In [1]:
%pip install -q kingdon anywidget==0.9.13

Note: you may need to restart the kernel to use updated packages.


# The 
# Willing `Kingdon` Clifford
# Algebra Library

**<p style="text-align: right;">Martin Roelfs</p>**
<p style="text-align: right;">Flanders Make, University of Antwerp</p>

In [2]:
%%js
require(
    {
      // it makes sense to wait a little bit when you are loading
      // reveal from a cdn in a slow connection environment
      waitSeconds: 15
    },
    [
      "https://unpkg.com/reveal.js@5.1.0/dist/reveal.js",
      "https://unpkg.com/reveal.js@5.1.0/plugin/notes/notes.js"
    ],

    function(Reveal, RevealNotes){
        function broadcast(event) {
            var cur = Reveal.getIndices();
            console.log('current', cur);
            var iFrames = [...document.querySelectorAll('iframe')].filter(x=>x.src);
            iFrames.forEach(iFrame=>iFrame.contentWindow.postMessage([cur.h, cur.v, cur.f]+'',"*"));
            
        };
        
        Reveal.addEventListener('fragmentshown', broadcast);
        Reveal.addEventListener('fragmenthidden', broadcast);
        Reveal.addEventListener('slidetransitionend', broadcast);
    }
);

<IPython.core.display.Javascript object>

In [3]:
import reveal_widgets

fragment_widget = reveal_widgets.FragmentWidget()
fragment_widget

FragmentWidget(state=[0, 0])

In [4]:
import ipywidgets as ipy

from animations import (
    graph_mechanism_func,
)
from animations.config import alg2d, options, animated_options

from animations.mechanism import tangent_widget, recompute_widget, mechanism_widget

## Introduction

- Why another GA library?
  - Design Philosophy
  - Basic Examples
- Inner Workings
- Industrial Examples
- Get started

## Why Another GA Library

- Python is very popular with the scientific community
  - ease of use
  - rapid prototyping
  - rich ecosystem of (scientific) tooling

- Several python GA libraries already exist
  - `clifford` numerical GA package
  - `galgebra` symbolic package
  - `tfga` tensor-flow package
  - `numga` JAX/numpy backends

These libraries are all very good within their area of specialization. However, to truly match the principles of Python, it should be possible to incrementally improve your algorithms from symbolic to numeric to arrays to PyTorch tensors and beyond.

## `kingdon` Design Philosophy

`kingdon` was developed with the following goals in mind:
- Easy to use API.
- Rapid prototyping.
  - Visualization: `ganja.js` enabled graphics in jupyter notebooks.
- Input agnostic: symbols, floats, tensors, etc.
  - If it supports $+, -, *$ and optionally $/$ and $\sqrt{}$ then it is a valid coefficient for a multivector.
- Performance: symbolic code generation and just-in-time compilation.

> **Add GA to any workflow**


## Basic Examples

## Inner Workings

Let's explain the inner workings of `kingdon` by means of a simple example:
Within $\mathbb{R}_{2, 0, 1}$ (2DPGA) consider the inner product between a bivector $B$ and a vector $v$:
$$ w = B \cdot v $$

To perform this computation symbolically with `kingdon` looks as follows:
```python
>>> from kingdon import Algebra
>>> 
>>> alg = Algebra(2, 0, 1)
>>> B = alg.bivector(name='B')
>>> v = alg.vector(name='v')
>>> B | v
(B01*v1 + B02*v2) 𝐞₀ + (B12*v2) 𝐞₁ + (-B12*v1) 𝐞₂
```

## Inner Workings

- Binary representation of basis blades:
  <table style="border-collapse: collapse; border:1px solid black">
  <tr style="border-collapse: collapse; border:1px solid black">
      <th style="">blades</th>
      <th style="background-color:#88FF88">1</th>
      <th style="background-color:#CCCCFF">𝐞₀</th>
      <th style="background-color:#CCCCFF">𝐞₁</th>
      <th style="background-color:#CCCCFF">𝐞₂</th>
      <th style="background-color:#FFCCCC">𝐞₀₁</th>
      <th style="background-color:#FFCCCC">𝐞₀₂</th>
      <th style="background-color:#FFCCCC">𝐞₁₂</th>
      <th style="background-color:#FFCCFF">𝐞₀₁₂</th>
    </tr>
    <tr style="border-collapse: collapse; border:1px solid black">
      <th style="">keys</th>
      <td style="background-color:#EEFFEE">000</td>
      <td style="background-color:#EEFFEE">001</td>
      <td style="background-color:#EEFFEE">010</td>
      <td style="background-color:#EEFFEE">100</td>
      <td style="background-color:#EEFFEE">011</td>
      <td style="background-color:#EEFFEE">101</td>
      <td style="background-color:#EEFFEE">110</td>
      <td style="background-color:#EEFFEE">111</td>
    </tr>
  </table>
- `kingdon` multivectors are mappings of key/value pairs:
    ```python
    >>> v = alg.vector(name='v')
    >>> v.keys()
    (1, 2, 4)
    >>> v.values()
    [v0, v1, v2]
    ```

In [5]:
from kingdon import Algebra
alg = Algebra(2, 0, 1)

colors = ['#88FF88', '#CCCCFF', '#FFCCCC', '#FFCCFF']

table = '<tr style="border-collapse: collapse; border:1px solid black">\n'
table += f'  <th style="">blades</th>\n'
for k, v in alg.canon2bin.items():
    table += f'  <th style="background-color:{colors[len(k) - 1]}">{alg._bin2canon_prettystr[v]}</th>\n'
table += '</tr>\n'
table += '<tr style="border-collapse: collapse; border:1px solid black">\n'
table += f'  <th style="">keys</th>\n'
for v in alg.canon2bin.values():
    table += f'  <td style="background-color:#EEFFEE">{bin(v)[2:].zfill(3)}</td>\n'
table += '</tr>'
# table += '<tr>\n'
# for v in alg.canon2bin.values():
#     table += f'  <td style="background-color:#EEFFEE">{v}</td>\n'
# table += '</tr>'
# print(table)

# x = alg.vector(name='x')
# x.values()


B = alg.bivector(name='B')
v = alg.vector(name='v')
B | v
# import inspect
# print(inspect.getsource(alg.ip[B.keys(), v.keys()][1]))
# keys_out, func = alg.ip[B.keys(), v.keys()]
# alg.mutivector(
#     keys=keys_out,
#     values=func(B.values(), v.values())
# )

(B01*v1 + B02*v2) 𝐞₀ + (B12*v2) 𝐞₁ + (-B12*v1) 𝐞₂

## Inner Workings

The `kingdon` internals are lazy: code is only generated once it is needed.
```python
>>> alg = Algebra(2, 0, 1)
>>> alg.ip
OperatorDict(codegen=<function codegen_ip at 0x0000025BFE604DC0>, ..., operator_dict={})
```
Executing `B | v` will cause the inner-product code to be generated and excecuted with `B` and `v` as input. Now the generated code will be stored and re-used next time:
```python
>>> alg.ip
OperatorDict(codegen=<function codegen_ip at 0x0000025BFE604DC0>, ..., operator_dict={((3, 5, 6), (1, 2, 4)): ((1, 2, 4), <function codegen_ip_112_x_14 at 0x0000025BFFCF7880>)})
```
The generated code is 
```python
def codegen_ip_112_x_14(A, B):
    [a01, a02, a12] = A
    [b0, b1, b2] = B
    return [a01*b1+a02*b2, a12*b2, -a12*b1]
```
So we see that *`kingdon` uses the "sparsity" of the input* and performs *symbolic optimization*.

In [6]:
# `B | v` is equivalent to
# ```python
# keys_out, func = alg.ip[B.keys(), v.keys()]
# alg.multivector(keys=keys_out, values=func(B.values(), v.values()))
# ```

## Inner Workings

Advanced customization:
- `graded` mode to reduce the number of types.
  - In the future this will be expanded to a more advanced typing system.
- `cse` to eliminate common subexpressions
- `wrapper` function to decorate the generated code with, e.g.
   ```python
   @numba.njit
   def codegen_ip_112_x_14(A, B):
       [a01, a02, a12] = A
       [b0, b1, b2] = B
       return [a01*b1+a02*b2, a12*b2, -a12*b1]
   ```
- `simp_func` is a filter function that is applied after every call, e.g. `sympy.simplify` in symbolic mode or `lambda x: abs(x) > 1e-9` for numerical input.
- `symbolcls`/`codegen_symbolcls` specify the symbol class to use during codegen and when making symbolic multivectors.

# Industrial Examples

Flanders Make is a strategic research centre for the make industry in the Flanders region of Belgium.

I'd like to share with you the usage of `kingdon` in two projects at Flanders Make:
- Aandrijflijn Concept Optimalisatie (**AnCoOpt**). Goal: [...] to develop [...] tools and methods to convert a customer request into an optimal machine concept for electrically driven positioning applications. A machine concept is optimal when it allows to minimize the machine component costs, energy consumption, material use in further detailed design and at the same time maximize the performance (speed, precision, etc.).
- Tolerance Design Optimization (**ToleDO**). Goal: provide a novel workflow and toolchain to enable engineers to obtain the best performance of their designs at a minimal production cost, by showing the impact of key manufacturing tolerances on functional performance early in the design phase. 

## AnCoOpt

Together with Michiel Haemers I am responsible for the concept generation.

- Use GA to generate topologies and initial coordinates.
- Use GA to calculate end effectuater positions.
- Look Away, Steven, Matrices: there will be some Linear Algebra.

> LAPyGAGA


## AnCoOpt

In [7]:
camera = alg2d.evenmv(e=1, e01=0.4, e02=0.2)

graph_mechanism = alg2d.graph(
    graph_mechanism_func,
    **options,
    height='300px',
    scale=6,
    camera=camera,
)
grid = ipy.GridspecLayout(9, 3, height='350px')
grid[0, 0] = mechanism_widget
grid[0, 1] = recompute_widget
grid[0, 2] = tangent_widget
grid[1:, :] = graph_mechanism
grid

GridspecLayout(children=(Dropdown(layout=Layout(grid_area='widget001'), options=(('Diamond', 0), ('Fivebar', 1…

## ToleDO

# Get Started

Just like `ganja.js` has its coffeeshop, `kingdon` has its teahouse.
(Ganja usage is tolerated in the Teahouse.)